<a href="https://colab.research.google.com/github/KSY1526/MyProject/blob/master/%EC%BA%90%EA%B8%80%209%EC%9B%94%20%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B8/kaggle6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
!git clone --recursive https://github.com/Microsoft/LightGBM
%cd /content/LightGBM
!mkdir build
!cmake -DUSE_GPU=1
!make -j$(nproc)
!sudo apt-get -y install python-pip
!sudo -H pip install setuptools pandas numpy scipy scikit-learn -U
%cd /content/LightGBM/python-package/
!sudo python setup.py install --precompile

Cloning into 'LightGBM'...
remote: Enumerating objects: 23473, done.
remote: Counting objects: 100% (1150/1150), done.
remote: Compressing objects: 100% (573/573), done.
remote: Total 23473 (delta 743), reused 885 (delta 557), pack-reused 22323
Receiving objects: 100% (23473/23473), 18.08 MiB | 22.28 MiB/s, done.
Resolving deltas: 100% (17136/17136), done.
Submodule 'include/boost/compute' (https://github.com/boostorg/compute) registered for path 'external_libs/compute'
Submodule 'eigen' (https://gitlab.com/libeigen/eigen.git) registered for path 'external_libs/eigen'
Submodule 'external_libs/fast_double_parser' (https://github.com/lemire/fast_double_parser.git) registered for path 'external_libs/fast_double_parser'
Submodule 'external_libs/fmt' (https://github.com/fmtlib/fmt.git) registered for path 'external_libs/fmt'
Cloning into '/content/LightGBM/external_libs/compute'...
remote: Enumerating objects: 21731, done.        
remote: Counting objects: 100% (3/3), done.        
remote: 

데이터 불러오기

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
train = pd.read_csv('/content/drive/MyDrive/train.csv')
test = pd.read_csv('/content/drive/MyDrive/test.csv')



데이터 탐색

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 957919 entries, 0 to 957918
Columns: 120 entries, id to claim
dtypes: float64(118), int64(2)
memory usage: 877.0 MB


In [ ]:
train

,id,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,...,f80,f81,f82,f83,f84,f85,f86,f87,f88,f89,f90,f91,f92,f93,f94,f95,f96,f97,f98,f99,f100,f101,f102,f103,f104,f105,f106,f107,f108,f109,f110,f111,f112,f113,f114,f115,f116,f117,f118,claim
0,0,0.108590,0.004314,-37.566,0.017364,0.289150,-10.25100,135.12,168900.0,3.992400e+14,86.489,0.59881,1.423200e+09,0.27240,9.455600,-0.050305,1938.3000,8.6331,4.060700,26.8670,-1.18000,10961.00,1.5397,135.3200,-1.496500,440.080,2.590100e+12,2.194200e+09,2968800.0,0.001431,13.3270,0.750500,18509.0,146820.0,-0.000276,1.090600e+16,1705.400,414.29,3.5392,1888.00,...,-0.001081,6.1244,1.231800e+11,275.9200,5308500.0,1704.000,5.022400e+10,53.39800,-2.2012,6871.00,3.8862,-0.005580,5252.100,166.6900,1.607400,0.66534,7768.900,0.99662,1.125700e+11,2.2432,0.934160,0.65056,94569.0,21.4710,8214.100,0.288010,0.097826,0.001071,1.412400e+09,0.110930,-12.2280,1.7482,1.90960,-7.11570,4378.80,1.2096,8.613400e+14,140.10,1.01770,1
1,1,0.100900,0.299610,11822.000,0.276500,0.459700,-0.83733,1721.90,119810.0,3.874100e+15,9953.600,1.20930,3.334100e+09,0.28631,-0.012858,-0.019912,10.2840,6.1872,1.041900,4.6404,31.87700,123620.00,1.3951,125.8100,1.198900,136.450,9.098100e+09,4.004100e+10,1564000.0,0.000204,3.1074,1.503300,238000.0,21440.0,-0.001344,3.079400e+16,229.100,844.82,1.4680,4726.50,...,0.254100,6.9191,1.832400e+11,9.6510,32800.0,1480.600,2.300600e+10,44.05100,205.6900,4295.30,13.3880,0.468430,754.610,83.2330,1.189000,29.55000,7343.700,0.99815,4.877700e+13,1.2708,-0.000969,5.29520,6779.0,227.7200,34.342,0.340300,0.143370,0.049276,1.903200e+09,0.976730,-56.7580,4.1684,0.34808,4.14200,913.23,1.2464,7.575100e+15,1861.00,0.28359,0
2,2,0.178030,-0.006980,907.270,0.272140,0.459480,0.17327,2298.00,360650.0,1.224500e+13,15827.000,0.38164,1.230300e+09,0.25807,2.455600,NaN,26.8730,7.5463,1.996700,1.9526,817.76000,-2948.70,2.0054,1.6826,1.196800,74.624,-3.273900e+10,5.718900e+10,11058.0,-0.003097,8.0241,1.131800,27940.0,862460.0,-0.002207,5.849100e+13,-897.840,NaN,1.3561,3063.40,...,0.260260,6.1052,1.013300e+11,357.2700,1476600.0,90.845,1.306200e+09,2.37310,391.3700,2965.30,NaN,0.494590,43.524,138.5200,1.107900,0.91948,47.915,NaN,1.510500e+12,3.4663,0.560950,4.13090,95531.0,39.4860,-83.148,0.084881,0.032222,0.001668,1.436500e+07,0.201020,-5.7688,1.2042,0.26290,8.13120,45119.00,1.1764,3.218100e+14,3838.20,0.40690,1
3,3,0.152360,0.007259,780.100,0.025179,0.519470,7.49140,112.51,259490.0,7.781400e+13,-36.837,1.10960,1.223100e+09,0.30944,10.370000,-0.106260,533.8400,7.8490,1.037900,8.0030,12.34900,-195.28,2.5598,92.1420,0.637890,1054.900,-1.204100e+10,5.187300e+12,1475400.0,1.036500,1.1903,0.989410,301200.0,NaN,-0.000007,-9.299200e+13,-10.818,1020.30,2.9553,3342.50,...,0.372830,1.5606,1.835400e+10,-3.4298,6485700.0,2120.000,3.081200e+10,34.05600,157.4300,3724.50,8.4211,0.407780,2971.200,204.7000,-0.979980,9.94050,12011.000,0.99898,5.063400e+13,1.2261,0.250200,0.72974,373690.0,194.6500,120.930,0.260710,0.234240,-0.002794,1.442300e+09,-0.011820,-34.8580,2.0694,0.79631,-16.33600,4952.40,1.1784,4.533000e+12,4889.10,0.51486,1
4,4,0.116230,0.502900,-109.150,0.297910,0.344900,-0.40932,2538.90,65332.0,1.907200e+15,144.120,1.05310,2.634100e+09,0.29782,2.654800,NaN,1808.9000,7.2783,3.975700,NaN,NaN,29520.00,3.4225,96.7250,0.797250,215.570,1.732600e+13,2.635200e+12,2161200.0,0.895470,6.8257,0.974130,142620.0,231350.0,0.001257,1.012500e+16,51.508,293.76,1.3351,3042.10,...,0.085690,1.5846,3.825200e+10,130.7000,102100.0,1951.800,1.142800e+10,58.56600,176.8300,1279.00,4.9662,0.479120,-70.278,10.8870,1.143400,6.19120,197.470,NaN,1.574800e+13,1.0083,0.339530,13.48600,201300.0,38.8420,324.000,0.238250,0.141550,0.002208,5.830700e+09,0.927390,-13.6410,1.5298,1.14640,-0.43124,3856.50,1.4830,-8.991300e+12,NaN,0.23049,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [ ]:
train.describe()

,id,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,...,f80,f81,f82,f83,f84,f85,f86,f87,f88,f89,f90,f91,f92,f93,f94,f95,f96,f97,f98,f99,f100,f101,f102,f103,f104,f105,f106,f107,f108,f109,f110,f111,f112,f113,f114,f115,f116,f117,f118,claim
count,957919.000000,942672.000000,942729.000000,942428.000000,942359.000000,942514.000000,942398.000000,942415.000000,9.425460e+05,9.426700e+05,942696.000000,942494.000000,9.423260e+05,942455.000000,942697.000000,942410.000000,942475.000000,942492.000000,942594.000000,942445.000000,942464.000000,942465.000000,942641.000000,942563.000000,942289.000000,942413.000000,9.425610e+05,9.424750e+05,9.426540e+05,942504.000000,942527.000000,942241.000000,9.423900e+05,9.424270e+05,942671.000000,9.425850e+05,942556.000000,942607.000000,942485.000000,942360.000000,...,942599.000000,942573.000000,9.424340e+05,942292.000000,9.425340e+05,942470.000000,9.423960e+05,942603.000000,942371.000000,942474.000000,942437.000000,942412.000000,942427.000000,942462.000000,942505.000000,942320.000000,942634.000000,942654.000000,9.426310e+05,942485.000000,942393.000000,942570.000000,9.427510e+05,942319.000000,942721.000000,942533.000000,942375.000000,942535.000000,9.425790e+05,942390.000000,942554.000000,942420.000000,942509.000000,942686.000000,942481.000000,942360.000000,9.423300e+05,942512.000000,942707.000000,957919.000000
mean,478959.000000,0.090201,0.345964,4068.744207,0.201214,0.304869,-0.071458,1620.843815,3.771642e+05,1.806054e+15,5323.442367,0.729518,1.844436e+09,0.247759,6.999152,0.019394,444.804439,6.892520,4.491982,22.448660,203.782031,61070.567499,2.269597,87.166891,0.341240,414.983986,3.379988e+12,1.254164e+12,2.256887e+06,0.329054,7.882455,0.394211,1.344700e+05,3.579055e+05,-0.000005,2.782318e+16,185.485192,406.072571,1.768743,1981.074344,...,0.194237,3.240370,1.053811e+11,152.850973,6.128694e+06,635.301741,3.251414e+10,26.598567,207.260994,3805.100719,6.733556,0.366754,4868.608259,132.264261,0.821190,13.120677,3849.687894,0.999970,1.415684e+13,1.683151,0.425840,20.213776,3.215760e+05,548.647493,3855.616303,0.178027,0.160832,0.014196,1.671873e+09,0.398622,-19.926398,2.074530,23.885245,1.748777,63152.973540,1.208876,4.276905e+16,3959.204669,0.559267,0.498492
std,276527.540591,0.043564,0.146251,6415.829440,0.212510,0.145343,2.123777,1276.281403,3.454325e+05,2.335204e+15,10068.380032,1.495535,2.125478e+09,0.101156,6.621472,0.101880,631.327086,1.715735,3.899241,14.609968,281.015052,89859.616863,0.896001,37.350264,1.643595,314.822410,5.655524e+12,1.642736e+12,2.303909e+06,0.433861,5.939864,0.781932,2.036605e+05,4.626670e+05,0.001534,3.452707e+16,701.769083,314.708639,0.588984,1958.688237,...,0.162364,1.992068,9.896094e+10,227.888751,8.767985e+06,583.468704,3.069150e+10,25.454605,158.215276,3532.626437,3.158678,0.146413,8428.424099,110.058170,0.712584,12.740238,6437.386650,0.001535,1.640385e+13,0.712016,0.283651,19.617660,3.277370e+05,864.175628,6670.256245,0.123319,0.141616,0.016259,1.875733e+09,0.298071,18.578439,0.895793,45.581360,10.088848,92435.016241,0.114959,6.732441e+16,3155.991777,0.408426,0.499998
min,0.000000,-0.149910,-0.019044,-9421.700000,-0.082122,-0.006990,-12.791000,-224.800000,-2.984300e+04,-1.153300e+15,-26404.000000,-8.086300,-4.081000e+08,-0.103800,-0.853760,-0.335660,-116.880000,-3.664500,-0.066527,-4.422500,-58.834000,-84079.000000,-6.009400,-20.514000,-5.735200,-71.502000,-6.956700e+11,-9.384200e+11,-4.706000e+05,-0.005659,-0.529990,-3.813500,-3.496500e+05,-6.055900e+05,-0.003881,-2.068900e+16,-2414.300000,-40.881000,0.546100,-433.700000,...,-0.017615,0.964200,-7.345700e+10,-28.752000,-2.992000e+06,-74.545000,-5.949500e+09,-7.616400,-22.576000,-296.780000,-0.257570,-0.012238,-12829.000000,-12.922000,-3.293300,-1.352400,-7764.300000,0.996100,-5.714600e+12,0.608200,-0.034559,-4.294900,-2.277700e+05,-222.210000,-11581.000000,-0.029027,-0.066726,-0.007535,-5.877000e+08,-0.042355,-105.860000,0.27704

데이터 반응변수, 설명변수로 만들기

In [3]:
train_x = train.drop(['id','claim'], axis = 1)
train_y = train['claim']

데이터 탐색2

In [5]:
train['claim'].value_counts()

0    480404
1    477515
Name: claim, dtype: int64

In [4]:
sum(train['f1'].isna())

15247

na값이 얼마나 있는지, na값과 claim값이 관련이 있는지 탐색

In [6]:
train[train['f1'].isna()]['claim'].value_counts()

1    11212
0     4035
Name: claim, dtype: int64

In [7]:
features = train_x.columns.values.tolist()
for feature in features:
    print(train[train[feature].isna()]['claim'].value_counts())

1    11212
0     4035
Name: claim, dtype: int64
1    11138
0     4052
Name: claim, dtype: int64
1    11458
0     4033
Name: claim, dtype: int64
1    11479
0     4081
Name: claim, dtype: int64
1    11370
0     4035
Name: claim, dtype: int64
1    11511
0     4010
Name: claim, dtype: int64
1    11527
0     3977
Name: claim, dtype: int64
1    11287
0     4086
Name: claim, dtype: int64
1    11291
0     3958
Name: claim, dtype: int64
1    11211
0     4012
Name: claim, dtype: int64
1    11510
0     3915
Name: claim, dtype: int64
1    11499
0     4094
Name: claim, dtype: int64
1    11465
0     3999
Name: claim, dtype: int64
1    11281
0     3941
Name: claim, dtype: int64
1    11489
0     4020
Name: claim, dtype: int64
1    11474
0     3970
Name: claim, dtype: int64
1    11491
0     3936
Name: claim, dtype: int64
1    11377
0     3948
Name: claim, dtype: int64
1    11541
0     3933
Name: claim, dtype: int64
1    11479
0     3976
Name: claim, dtype: int64
1    11411
0     4043
Name: claim, dtype

=> 결측치도 하나의 반응변수로 만들어야겠다고 판단.

결측치를 하나의 반응변수(f119)로 만들기.




In [10]:
train_x_na = train_x.isna().astype('int')
train_x['f119'] = train_x_na['f1']
for feature in features:
  train_x['f119'] = train_x['f119'] + train_x_na[feature]

train_x['f119'] = train_x['f119'] - train_x_na['f1']
train_x['f119'][train_x['f119'] > 0] = 1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [11]:
test_na = test.isna().astype('int')
test['f119'] = test_na['f1']
for feature in features:
  test['f119'] = test['f119'] + test_na[feature]

test['f119'] = test['f119'] - test_na['f1']
test['f119'][test['f119'] > 0] = 1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


결측치를 독립적인 반응변수로 만들었음으로 평균값으로 채워놓기

In [12]:
for feature in features:
    train_x[feature] = train_x[feature].fillna(train_x[feature].mean())
    test[feature] = test[feature].fillna(train_x[feature].mean())

본격적인 분석시작. 트레인/테스트 데이터로 분리

In [13]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(train_x, train_y, test_size = 0.2, random_state=42)

lightgbm 방법 이용. 하이퍼 파라미터 조정

In [31]:
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
lgbm_clf = LGBMClassifier(max_depth = 13,reg_alpha = 10, reg_lambda  = 15,random_state=42)# num_leaves=130,

params = {
    'num_leaves' : [110,120,130,140]
}

gridcv = GridSearchCV(lgbm_clf, param_grid=params, cv = 3)
gridcv.fit(x_train, y_train)
print(gridcv.best_params_)
print(roc_auc_score(y_test,gridcv.predict_proba(x_test)[:,1]))

{'num_leaves': 130}
0.7964663924504807


In [32]:
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
lgbm_clf = LGBMClassifier(n_estimators = 1000, learning_rate = 0.01, max_depth = 13, num_leaves=130, reg_alpha = 10, reg_lambda  = 15)
lgbm_clf.fit(x_train, y_train )


pred = lgbm_clf.predict_proba(x_test)
print(roc_auc_score(y_test,pred[:,1]))
pred2 = lgbm_clf.predict_proba(x_train)
print(roc_auc_score(y_train,pred2[:,1]))

0.7983878057353571
0.8519066948920782


트레인데이터 전체로 모델에 fit 시킨후 테스트 데이터에 적용 후 파일 생성

In [33]:
lgbm_clf = LGBMClassifier(n_estimators = 1000, learning_rate = 0.01, max_depth = 13, num_leaves=130, reg_alpha = 10, reg_lambda  = 15)
lgbm_clf.fit(train_x, train_y)
pred = lgbm_clf.predict_proba(test.iloc[:,1:])
test['claim'] = pred[:,1]
final = test[['id', 'claim']]
final.to_csv('/content/drive/MyDrive/kaggle9_6.csv',encoding='UTF-8', index=False)

ImportError: ignored